In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/Llama-3-Open-Ko-8B")
model = AutoModelForCausalLM.from_pretrained("beomi/Llama-3-Open-Ko-8B")


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

LangChain에서 Hugging Face 모델을 연결하기 위해, 먼저 Transformers 라이브러리로 모델을 로드한 뒤 pipeline 객체를 생성합니다.

In [33]:
from transformers import pipeline
from langchain import HuggingFacePipeline

# 파이프라인 생성
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=50,  # 더 짧은 응답을 위해 max_length 줄이기
    temperature=0.5,  # 낮은 온도로 생성의 예측성 높이기
    top_p=0.9,
    do_sample=True,
    device=0
)
# HuggingFacePipeline으로 래핑
llama_chain = HuggingFacePipeline(pipeline=hf_pipeline)

llama_chain.invoke("What is stock?")


"What is stock?(이름) '대통령, 야당과 소통해야…대통령이 먼저 나서야'\n= (이름) 국민의당 전 상임 공동대표는 16일 '대"

In [36]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 민봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. "),
    ("user", "{input}")
])

chain = prompt | llama_chain
chain.invoke({"input": "LLM이 뭐야?"})

'System: You are a helpful, professional assistant named 민봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. \nHuman: LLM이 뭐야? AI야? AI야? AI야'

In [35]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llama_chain| StrOutputParser()

chain.invoke({"input": "대한민국 수도가 어디야?"})

'System: You are a helpful, professional assistant named 민봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. \nHuman: 대한민국 수도가 어디야? \nSystem: 대한민국 수도는 서울특별시입니다'

In [21]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [22]:
# 사용할 Few-shot예제를 정의합니다.
# 여기서도 마찬가지로 뉴스 제목 생성에 관해 진행하겠습니다.
examples = [
    {
        "domain": "경제",
        "answer": "ESG 투자 확산, 기업들이 새로운 경영 패러다임을 맞이하다",
    },
    {
        "domain": "정치",
        "answer": "정당 간 연합 논의 본격화, 정치 지형에 미칠 영향은",
    },
    {
        "domain": "과학",
        "answer": "유전자 편집 기술의 미래: CRISPR의 혁신과 윤리적 논의"
    },
]

# 입력할 도메인, 그리고 답변을 PromptTemplate에 입력합니다.
example_prompt = PromptTemplate.from_template(
    "Domain:\n{domain}\nAnswer:\n{answer}"
)
print(example_prompt.format(**examples[0]))

Domain:
경제
Answer:
ESG 투자 확산, 기업들이 새로운 경영 패러다임을 맞이하다


In [23]:
### TODO
# example, example_prompt, input_variables를 활용해주세요
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Domain:\n{domain}\nAnswer:",
    input_variables=["domain"],
)
###

domain = "환경"
final_prompt = prompt.format(domain=domain)
print(final_prompt)

Domain:
경제
Answer:
ESG 투자 확산, 기업들이 새로운 경영 패러다임을 맞이하다

Domain:
정치
Answer:
정당 간 연합 논의 본격화, 정치 지형에 미칠 영향은

Domain:
과학
Answer:
유전자 편집 기술의 미래: CRISPR의 혁신과 윤리적 논의

Domain:
환경
Answer:


In [25]:
# 결과 출력
answer = llama_chain.invoke(final_prompt)
print(answer)

Domain:
경제
Answer:
ESG 투자 확산, 기업들이 새로운 경영 패러다임을 맞이하다

Domain:
정치
Answer:
정당 간 연합 논의 본격화, 정치 지형에 미칠 영향은

Domain:
과학
Answer:
유전자 편집 기술의 미래: CRISPR의 혁신과 윤리적 논의

Domain:
환경
Answer:이번에 새로 나온 신제품입니다.
Dies ist ein neues Produkt.
국내에서 코로나19 백신 접종이 시작된 지 100일을 맞았다. 2일 코로나19 예방접종대응추진단에 따르면 이날 0시 기준 누적 접종자는 1,000만 2,000명이다. 전 국민의 19.5%가 1차 접종을 마쳤고, 2차 접종
